In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

In [22]:
df = pd.read_csv("diabetes.csv")
# df = df.drop(['Column1', 'Column2', 'Column3', 'Column4'], axis=1)

In [23]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [24]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']


standard_scaler = StandardScaler()
X_standard_scaled = standard_scaler.fit_transform(X)

df_standard_scaled = pd.DataFrame(X_standard_scaled, columns=X.columns)

print("🔵 Standard Scaled Data:")
print(df_standard_scaled)

🔵 Standard Scaled Data:
     Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin       BMI  \
0       0.639947  0.848324       0.149641       0.907270 -0.692891  0.204013   
1      -0.844885 -1.123396      -0.160546       0.530902 -0.692891 -0.684422   
2       1.233880  1.943724      -0.263941      -1.288212 -0.692891 -1.103255   
3      -0.844885 -0.998208      -0.160546       0.154533  0.123302 -0.494043   
4      -1.141852  0.504055      -1.504687       0.907270  0.765836  1.409746   
..           ...       ...            ...            ...       ...       ...   
763     1.827813 -0.622642       0.356432       1.722735  0.870031  0.115169   
764    -0.547919  0.034598       0.046245       0.405445 -0.692891  0.610154   
765     0.342981  0.003301       0.149641       0.154533  0.279594 -0.735190   
766    -0.844885  0.159787      -0.470732      -1.288212 -0.692891 -0.240205   
767    -0.844885 -0.873019       0.046245       0.656358 -0.692891 -0.202129   

     DiabetesPe

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    df_standard_scaled,  # features
    y,                   # target
    test_size=0.2,       # 20% for testing
    random_state=42,     # for reproducibility
    stratify=y           # keep the same proportion of classes
)

# Optional: print the shapes of the splits
print("🔹 Training set shape:", X_train.shape)
print("🔹 Test set shape:", X_test.shape)

🔹 Training set shape: (614, 8)
🔹 Test set shape: (154, 8)


In [26]:
from tensorflow.keras.layers import BatchNormalization

model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


In [27]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [28]:
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1)

Epoch 1/100
62/62 [==============================] - 1s 1ms/step - loss: 1.1033 - accuracy: 0.4072
Epoch 2/100
62/62 [==============================] - 0s 1ms/step - loss: 0.7249 - accuracy: 0.5733
Epoch 3/100
62/62 [==============================] - 0s 1ms/step - loss: 0.6706 - accuracy: 0.6336
Epoch 4/100
62/62 [==============================] - 0s 1ms/step - loss: 0.5436 - accuracy: 0.7199
Epoch 5/100
62/62 [==============================] - 0s 2ms/step - loss: 0.5808 - accuracy: 0.7068
Epoch 6/100
62/62 [==============================] - 0s 1ms/step - loss: 0.5927 - accuracy: 0.6938
Epoch 7/100
62/62 [==============================] - 0s 1ms/step - loss: 0.5573 - accuracy: 0.7362
Epoch 8/100
62/62 [==============================] - 0s 1ms/step - loss: 0.5373 - accuracy: 0.7264
Epoch 9/100
62/62 [==============================] - 0s 1ms/step - loss: 0.5578 - accuracy: 0.7590
Epoch 10/100
62/62 [==============================] - 0s 1ms/step - loss: 0.5284 - accuracy: 0.7296
Epoch 11/

In [29]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))


5/5 [==============================] - 0s 1ms/step
[[83 17]
 [21 33]]
              precision    recall  f1-score   support

           0       0.80      0.83      0.81       100
           1       0.66      0.61      0.63        54

    accuracy                           0.75       154
   macro avg       0.73      0.72      0.72       154
weighted avg       0.75      0.75      0.75       154



In [30]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

5/5 [==============================] - 0s 2ms/step - loss: 0.5137 - accuracy: 0.7532
Test Accuracy: 0.7532
